In [35]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None #'<password>'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

#r.flushdb()

Connected to Redis


# TimeSeries Data Type

## Description of the use case
This notebook contains examples for Redis time-series data type.
The Redis time series data type stores real-valued data points along with the time they were collected. We can combine the values from a selection of time series and query them by time or value range. We can also compute aggregate functions of the data over periods of time and create new time series from the results. When you create a time series, you can specify a maximum retention period for the data, relative to the last reported timestamp, to prevent the time series from growing indefinitely.

Time series support very fast reads and writes, making them ideal for applications such as:

- Instrument data logging
- System performance metrics
- Financial market data
- Internet of Things (IoT) sensor data
- Smart metering
- Quality of service (QoS) monitoring


## Create a simple timeseries data


In [36]:
import time
import random

# Ticker key
price_key = "ticker:RELIANCE:price"
rts=r.ts()

# Delete if exists
if r.exists(price_key):
    r.delete(price_key)

# Create time series with stock-related labels
rts.create(price_key, labels={"ticker": "RELIANCE", "market": "NSE", "type": "price"})

# Insert 1000 price points, simulating 1-second intervals
start_time = int(time.time()) * 1000
base_price = 2800.0

print("Inserting simulated price data for RELIANCE...")
printable_ts = start_time + 9997*1000
for i in range(10000):
    timestamp = start_time + i * 1000
    price = base_price + random.uniform(-2, 2)  # Simulated price fluctuation
    rts.add(price_key, timestamp, round(price, 2))
    if(timestamp > printable_ts):
        print("ts ="+str(timestamp) +"  price ="+str(price))

print("Inserted 10000 price data points.")


Inserting simulated price data for RELIANCE...
ts =1754412254000  price =2801.2743019219056
ts =1754412255000  price =2799.1996846136985
Inserted 10000 price data points.


## Get Latest Data

In [37]:
latest_price = rts.get(price_key)
print(f"Latest RELIANCE Price: {latest_price}")

Latest RELIANCE Price: (1754412255000, 2799.2)


## Price History Over a Time Range

In [61]:
# Fetch first 10 minutes of data
from_ts = start_time
to_ts = start_time + 600 * 1000  # 10 minutes

price_history = rts.range(price_key, from_ts, to_ts)
print(f"Fetched {len(price_history)} price points.")
print(price_history[:5])


Fetched 601 price points.
[(1754402256000, 2799.9), (1754402257000, 2799.42), (1754402258000, 2801.72), (1754402259000, 2801.99), (1754402260000, 2798.07)]


## TS.RANGE with Aggregation (1-Min Avg)

In [39]:
# Average RELIANCE price per minute
avg_per_min = rts.range(price_key, from_ts, to_ts, aggregation_type="avg", bucket_size_msec=60_000)
print("RELIANCE 1-minute average price (first 5 buckets):")
print(avg_per_min)



RELIANCE 1-minute average price (first 5 buckets):
[(1754402220000, 2800.4841666666666), (1754402280000, 2799.926833333332), (1754402340000, 2799.6175000000003), (1754402400000, 2800.0926666666664), (1754402460000, 2800.0175000000013), (1754402520000, 2800.076833333333), (1754402580000, 2800.1181666666675), (1754402640000, 2799.947666666668), (1754402700000, 2799.926166666666), (1754402760000, 2799.9431666666665), (1754402820000, 2800.011351351351)]


## Get Latest Price Using Labels

In [40]:
# Fetch latest prices for all NSE tickers (or just RELIANCE here)
#latest_prices = rts.mget(["market=NSE"])
latest_prices = rts.mget(["ticker=RELIANCE"])
for entry in latest_prices:
    print(entry)
    print(f"{entry['ticker:RELIANCE:price']}")


{'ticker:RELIANCE:price': [{}, 1754412255000, 2799.2]}
[{}, 1754412255000, 2799.2]


## Add Volume data


In [41]:
volume_key = "ticker:RELIANCE:volume"

# Create volume time series
if r.exists(volume_key):
    r.delete(volume_key)

rts.create(volume_key, labels={"ticker": "RELIANCE", "market": "NSE", "type": "volume"})

# Insert volume data corresponding to price timestamps
print("Inserting volume data...")

for i in range(10000):
    timestamp = start_time + i * 1000
    volume = random.randint(1000, 5000)
    rts.add(volume_key, timestamp, volume)

print("Volume data inserted.")


Inserting volume data...
Volume data inserted.


## Query Both Price and Volume

In [47]:
# Fetch data for RELIANCE (both price & volume)
mr_results = rts.mrange(from_ts, to_ts, filters=["ticker=RELIANCE"])

price_list = mr_results[0]['ticker:RELIANCE:price'][1]
volume_list = mr_results[1]['ticker:RELIANCE:volume'][1]

timestamp = 1754402263000
price_dict = dict(price_list)
volume_dict = dict(volume_list)
#print(price_dict)
#print(volume_dict)
price = price_dict.get(timestamp)
volume = volume_dict.get(timestamp)
print("price=" + str(price))
print("volume=" + str(volume))


price=2801.0
volume=3713.0


## Search

In [60]:
# Example 1: Get all tickers on NSE
nse_data = rts.mrange(from_ts, to_ts, filters=["market=NSE"])

# Example 2: Get only RELIANCE price
reliance_price_data = rts.mrange(from_ts, to_ts, filters=["ticker=RELIANCE", "type=price"])

# Example 3: All volume series
volume_data = rts.mrange(from_ts, to_ts, filters=["type=volume"])
#print(nse_data)
print(f"NSE tickers count: {len(nse_data)}") # price and volume
#print(type(reliance_price_data))
print(len(reliance_price_data[0]['ticker:RELIANCE:price'][1]))
print(f"Volume series data (first element): {(volume_data[0]['ticker:RELIANCE:volume'][1])[0]}")


NSE tickers count: 2
601
Volume series data (first element): (1754402256000, 4419.0)
